# <font color = 'navy'>Pruning Fungal tree and building corresponding codon usage trait table for downstream Phylogenetic Comparative analysis in R</font> 

#### Author: Rhondene Wint

### Background:
This notebooks contains python code for prepare the fungal data for phylogenetic comparative analyses in R using the phytools suite. Yes, I am sure this can be done in R too.

In [3]:
import pandas as pd
#phylopackages
import ete3
import dendropy
import Bio
from Bio.Phylo import  NewickIO

import toytree       # a tree plotting library
import toyplot       # a general plotting library

In [4]:
##load table with codon frequency, codon bias (ENC) and GC3%
df_codon_usage=pd.read_csv('combined_codon_usage_ratios_Updated.csv')
df_codon_usage.head()

,Ala-GCA,Ala-GCC,Ala-GCG,Ala-GCU,Arg2-AGG,Arg2-AGA,Arg4-CGA,Arg4-CGC,Arg4-CGG,Arg4-CGU,...,Thr-ACU,Tyr-UAC,Tyr-UAU,Val-GUA,Val-GUC,Val-GUG,Val-GUU,Species,Mean_ENC,Mean_GC3
0,0.259876,0.289691,0.208468,0.241965,0.540714,0.459286,0.272465,0.349953,0.167018,0.210564,...,0.214809,0.618371,0.381629,0.138147,0.382033,0.243651,0.236170,Aaoar1,53.380551,0.571015
1,0.260036,0.235526,0.131363,0.373075,0.354074,0.645926,0.333440,0.181087,0.150080,0.335393,...,0.312145,0.396400,0.603600,0.183287,0.254893,0.332925,0.228895,Absrep1,50.651273,0.380234
2,0.206847,0.384653,0.233511,0.174988,0.610441,0.389559,0.218941,0.403203,0.235758,0.142098,...,0.120681,0.758989,0.241011,0.059821,0.536751,0.280313,0.123115,Acain1,46.339993,0.700730
3,0.274505,0.232841,0.185094,0.307560,0.428325,0.571675,0.369957,0.256638,0.168074,0.205332,...,0.253408,0.547403,0.452597,0.136288,0.348398,0.226146,0.289168,Acema1,55.429611,0.505663
4,0.130164,0.452521,0.247264,0.170051,0.630795,0.369205,0.139417,0.398381,0.332124,0.130078,...,0.141471,0.695521,0.304479,0.076299,0.472172,0.302366,0.149163,Aciaci1,45.924838,0.722603


In [ ]:
#load the complete Fungal treee obtained from JGI Mycocosm
big_tree = dendropy.Tree.get(
        path="../phylo_fungi_R/fungi_2020_public_fixed.newik",
        schema="newick", rooting='default-rooted' )

In [ ]:
#Prune the large tree to retain only species from the codon usage dataset

species=df_codon_usage['Species'].values  # get species name from the codon usage table
big_tree = dendropy.Tree.get(path="../phylo_fungi_R/fungi_2020_public_fixed.newik",
                                 schema="newick", rooting='default-rooted' )
##keep rooted 
big_tree.retain_taxa_with_labels(species)
big_tree.update_bipartitions()
## save to file
with open(f'../phylo_fungi_R/all_codon_fracsUpdated.newick', "w") as f:
    f.write(big_tree.as_string("newick"))

### <font color ='red'>To build the trait table that matches the species order of the pruned tree,  make a copy of the newick file of the pruned tree, remove the [&R] in notepad/text editor and save a copy or else it won't load in python </font>
- NOTE that you need the newick version of the rooted pruned tree to do downstream analysis in R, hence why you save a separate copy. 

In [ ]:
##Build trait data table as same order as leaf names in pruned tree
pruned_tree = ete3.Tree(f'../phylo_fungi_R/all_codon_fracsUpdated_Copy.newick',format=1)
pruned_leaves = [leaf.name for leaf in pruned_tree]
dfs=[]
for species in pruned_leaves:
    df = sample[sample['Species']==species]
    dfs.append(df)
traits = pd.concat(dfs)
traits.to_csv('../phylo_fungi_R/all_codon_frac_traitsUpdated.csv',sep=',',index=False)
traits.head()

## You now have a pruned tree and associated trait table. Head over to R for PCM analysis